In [1]:
%pip install --upgrade pip

%pip install requests pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import pandas as pd

In [2]:
def load_api_key(api_key_file_path):
    with open(api_key_file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

In [3]:
github_pat = github_pat = load_api_key('github-pat.txt')
headers = {'Authorization': 'token ' + github_pat}
owner = "subodh30"

In [4]:
def fork_repository(repo_url):
    repo_identifier = repo_url.split('github.com/')[1]
    response = requests.post(f'https://api.github.com/repos/{repo_identifier}/forks', headers=headers)
    print(response)
    if response.status_code == 202:
        print(f'Forked repository: {repo_url}')
    else:
        print(f'Failed to fork repository: {repo_url}')

In [5]:
def get_owners_repo_link(repo_url):
    repo_identifier = repo_url.split('github.com/')[1]
    repo_name = repo_identifier.split('/')[1]
    return owner + "/" + repo_name

In [6]:
def enable_issues(repo_url):
    repo_name = get_owners_repo_link(repo_url)
    data = {
        "has_issues": True,
    }
    response = requests.patch(f'https://api.github.com/repos/{repo_name}', headers=headers, json=data)

    if response.status_code == 200:
        print(f"Issues have been successfully enabled for repository {repo_name}.")
    else:
        print(f"Failed to enable issues for repository {repo_name}. Status code: {response.status_code}")
        print(response.text) 

In [7]:
def create_issue(repo_url, issue_title, issue_body):
    repo_name = get_owners_repo_link(repo_url)
    data = {
        'title': issue_title,
        'body': issue_body
    }
    response = requests.post(f'https://api.github.com/repos/{repo_name}/issues', headers=headers, json=data)
    if response.status_code == 201:
        print(f'Created issue in repository: {repo_name}')
    else:
        print(f'Failed to create issue in repository: {repo_name}')


In [13]:
fork_repository("https://github.com/apache/rocketmq")
enable_issues("https://github.com/apache/rocketmq")
create_issue("https://github.com/apache/rocketmq", "@devops-llm-bot", "Create github actions to build and test. Worlflow name must be - Generated Workflow - only.  add workflow dispatch event so that it can be trggered with api call. Only give yml file as a response and nothing else.")

<Response [202]>
Forked repository: https://github.com/apache/rocketmq
Issues have been successfully enabled for repository subodh30/rocketmq.
Created issue in repository: subodh30/rocketmq


In [6]:
def get_latest_PR_id(repo_url):
    repo_name = get_owners_repo_link(repo_url)
    params = {"state": "all", "sort": "created", "direction": "desc"}
    response = requests.get(f'https://api.github.com/repos/{repo_name}/pulls', headers=headers, params=params)
    if response.status_code == 200:
        prs = response.json()
        if prs:
            latest_pr = prs[0]
            return latest_pr['number']
        else:
            print("No open PRs found in the repository.")
    else:
        print(f"Failed to fetch PRs. Status code: {response.status_code}")
    return None

In [7]:
def status_check_of_PR(repo_url):
    repo_name = get_owners_repo_link(repo_url)
    pr_number = get_latest_PR_id(repo_url)

    while True:

        response = requests.get(f'https://api.github.com/repos/{repo_name}/pulls/{pr_number}/commits', headers=headers)
        print(response.json()[-1]["commit"]["url"])
        if response.status_code == 200:

            latest_commit = response.json()[-1]
            checks_url = latest_commit["commit"]["url"] + "/check-runs"
            checks_response = requests.get(checks_url, headers=headers)
            print(checks_response.json())
            if checks_response.status_code == 200:
                checks_data = checks_response.json()
                check_runs = checks_data.get("check_runs", [])

                # Check if all checks have completed successfully
                checks_passed = all(check_run["conclusion"] == "success" for check_run in check_runs)

                if checks_passed:
                    print("All checks have passed.")
                    break
                else:
                    print("Some checks are still in progress. Waiting for 30 seconds...")
                    time.sleep(30)
            else:
                print(f"Failed to get check runs. Status code: {checks_response.status_code}")
                break
        else:
            print(f"Failed to get PR commits. Status code: {response.status_code}")
            break

In [9]:
def run_workflow(repo_url, workflow_name):
    repo_name = get_owners_repo_link(repo_url)
    workflow_dispatch_payload = {
        "ref": "develop",  
        "inputs": {}  
    }
    workflow_dispatch_url = f"https://api.github.com/repos/{repo_name}/actions/workflows/{workflow_name}/dispatches"

    trigger_response = requests.post(workflow_dispatch_url, json=workflow_dispatch_payload, headers=headers)
    print(trigger_response.json())
    if trigger_response.status_code == 204:
        print(f"Workflow '{workflow_name}' has been triggered.")

        # Poll for the workflow run status
        while True:
            workflow_runs_url = f"https://api.github.com/repos/{repo_name}/actions/workflows/{workflow_name}/runs"
            runs_response = requests.get(workflow_runs_url, headers=headers)

            if runs_response.status_code == 200:
                runs_data = runs_response.json()
                latest_run = runs_data["workflow_runs"][0]
                status = latest_run["status"]
                conclusion = latest_run["conclusion"]

                print(f"Latest run status: {status}, Conclusion: {conclusion}")

                if status == "completed":
                    break
                elif status == "in_progress":
                    print("Workflow is still in progress. Waiting for 30 seconds...")
                    time.sleep(30)
                else:
                    print("Workflow run has encountered an error.")
                    break
            else:
                print(f"Failed to retrieve workflow run status. Status code: {runs_response.status_code}")
                break
    else:
        print(f"Failed to trigger the workflow. Status code: {trigger_response.status_code}")

In [14]:
# status_check_of_PR("https://github.com/apache/rocketmq")
run_workflow("https://github.com/apache/rocketmq", "build-autogenerated.yml")

{'message': "Workflow does not have 'workflow_dispatch' trigger", 'documentation_url': 'https://docs.github.com/rest/actions/workflows#create-a-workflow-dispatch-event'}
Failed to trigger the workflow. Status code: 422


In [ ]:
csv_files = ['DevOps_LLM_Bot_Test_Data - C#.csv', 'DevOps_LLM_Bot_Test_Data - Python.csv', 'DevOps_LLM_Bot_Test_Data - Java.csv']

for csv_file in csv_files:
    csvFile = pd.read_csv(csv_file)
    csvFile = csvFile[csvFile['GitHub_Repo_Link'].notna()]
    for i in range(0,len(csvFile)):
        repo_link = csvFile.iloc[i]['GitHub_Repo_Link']
        print(repo_link) 
        #fork_repository(repo_link)
        issue_title = '@devops-llm-bot create build and test github.'
        # enable_issues("https://github.com/apache/rocketmq")
        #create_issue(repo_link, issue_title)

In [9]:
def delete_repository(repo_url):
    repo_identifier = repo_url.split('github.com/')[1]
    repo_name = repo_identifier.split('/')[1]
    response = requests.delete(f'https://api.github.com/repos/{owner}/{repo_name}', headers=headers)

    if response.status_code == 204:
        print(f"Repository {owner}/{repo_name} has been successfully deleted.")
    else:
        print(f"Failed to delete the repository {owner}/{repo_name}. Status code: {response.status_code}")
        print(response.text)

In [10]:
delete_repository("https://github.com/apache/rocketmq")

Repository subodh30/rocketmq has been successfully deleted.
